In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.models.MatchLSTM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/96 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.409 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 96/96 [00:01<00:00, 49.23it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 91/91 [00:00<00:00, 192.09it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 8

In [4]:
preprocessor.context

{'embedding_input_dim': 318,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x2597e9dfac8>,
 'vocab_size': 318,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x2597e9ff278>}

In [5]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [7]:
padding_callback = mz.models.MatchLSTM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.MatchLSTM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchLSTM(
  (embedding): Embedding(318, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (input_proj): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(100, 100, bidirectional=True)
    )
  )
  (match_module): MatchModule(
    (v2_proj): Linear(in_features=200, out_features=200, bias=True)
    (proj): Linear(in_features=800, out_features=400, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (mlstm_module): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(400, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=200, out_features=200, bias=True)
    (2): Tanh()
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
) 996001


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-1 Loss-0.985]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-2 Loss-0.998]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-3 Loss-0.983]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-4 Loss-0.979]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-5 Loss-0.960]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-6 Loss-0.882]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-7 Loss-0.797]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-8 Loss-0.404]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-9 Loss-0.208]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818



[Iter-10 Loss-0.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1818 - normalized_discounted_cumulative_gain@5(0.0): 0.1818 - mean_average_precision(0.0): 0.1818

Cost time: 5.442551612854004s


In [11]:
import gc
gc.collect()

983